In [1]:
from utils import DevanagariString, VOWELS_GRADES, VOWELS_SET, VERB_GRADES_BEFORE_VOWELS, VERB_GRADES_BEFORE_CONSONANTS
from collections import defaultdict

In [2]:
DevanagariString("वद")+DevanagariString("आन्")+"मि", DevanagariString("")+"अन्"+"मि"

(वदआन्मि, अन्मि)

In [54]:
from abc import ABC, abstractmethod

class Verb(ABC):
    tenses = ["Present", "Imperfect", "Imperative", "Potential"]
    numbers = ["s", "d", "p"]
    persons = ["1", "2", "3"]

    prefixes = defaultdict(lambda: "")
    prefixes["Imperfect"] = "अ"
    suffixes = {
        "P": {
            "Present": {
                "1s": "आमि",
                "2s": "असि",
                "3s": "अति",
                "1d": "आवः",
                "2d": "अथ",
                "3d": "अतः",
                "1p": "आमः",
                "2p": "अथ",
                "3p": "अन्ति",
            },
            "Imperfect": {
                "1s": "अम्",
                "2s": "अः",
                "3s": "अत्",
                "1d": "आव",
                "2d": "अतम्",
                "3d": "अताम्",
                "1p": "आम",
                "2p": "अत",
                "3p": "अन्",
            },
            "Imperative": {
                "1s": "आनि", 
                "2s": "अ",
                "3s": "अतु", 
                "1d": "आव",# same as imperfect
                "2d": "अतम्", # same as imperfect
                "3d": "अताम्", # same as imperfect
                "1p": "आम", # same as imperfect
                "2p": "अत", # same as imperfect
                "3p": "अन्तु",
            },
            "Potential": { # same as imperfect, but with "य" inserted before the ending
                "1s": "एयम्",
                "2s": "ए:",
                "3s": "एत्",
                "1d": "एव",
                "2d": "एतम्",
                "3d": "एताम्",
                "1p": "एम",
                "2p": "एत",
                "3p": "एयुः", # different from imperfect
            },
            # TODO: future, aorist, perfect, etc.
        },
        "A" : {
            "Present": {
                "1s": "ए",
                "2s": "असे",
                "3s": "अते",
                "1d": "आवहि",
                "2d": "एथे",
                "3d": "एते",
                "1p": "आमहि",
                "2p": "आध्वम्",
                "3p": "अन्ते",
            },
            "Imperfect": {
                "1s": "ए",
                "2s": "अथाः",
                "3s": "अत",
                "1d": "आवहि",
                "2d": "एथाम्",
                "3d": "एताम्",
                "1p": "आमहि",
                "2p": "अध्वम्",
                "3p": "अन्त",
            },
            "Imperative": { 
                "1s": "ऐ",
                "2s": "अस्व",
                "3s": "अताम्",
                "1d": "आवहै",
                "2d": "एथाम्",
                "3d": "एताम्",
                "1p": "आमहै",
                "2p": "अध्वम्",
                "3p": "अन्ताम्",
            },
            "Potential": { # same as imperfect, but with "य" inserted before the ending
                "1s": "एय", 
                "2s": "एथाः",
                "3s": "एत",
                "1d": "एवहि",
                "2d": "एयाथाम्",
                "3d": "एयताम्",
                "1p": "एमहि",
                "2p": "एध्वम्",
                "3p": "एरन्", # different from imperfect
            },
            # TODO: future, aorist, perfect, etc.
        }
    }

    def __init__(self, root, verb_class, verb_p_or_a):
        self.root = root # eg. "वद्"
        self.verb_class = verb_class
        self.verb_p_or_a = verb_p_or_a.upper()

        assert verb_p_or_a in ["P", "A"]

    @abstractmethod
    def get_verb_base(self):
        pass

    @abstractmethod
    def get_verb_base(self):
        pass

    @abstractmethod
    def print_conjugation_tables(self, passive=False):
        pass

    def get_verb_grade(self, verb_grade):
        replacement_vowel = None
        for ix, char in enumerate(list(self.root)):
            if char in (VOWELS_GRADES):
                if (ix+1)>=len(self.root) or self.root[ix+1] in VOWELS_SET: # For thematic verbs if the last character is a vowel, the subsequent affix's first character is always a vowel
                    replacement_vowel = VERB_GRADES_BEFORE_VOWELS[char]
                else:
                    replacement_vowel = VERB_GRADES_BEFORE_CONSONANTS[char]
                break
        if not replacement_vowel:
            return DevanagariString(self.root)
        elif ix == len(self.root)-1:
            return DevanagariString(self.root[:ix]) + DevanagariString("्") + DevanagariString(replacement_vowel[verb_grade])
        else:
            return DevanagariString(self.root[:ix]) + DevanagariString("्") + DevanagariString(replacement_vowel[verb_grade]) + DevanagariString(self.root[ix+1:])

class ThematicVerb(Verb):

    def get_verb_base(self): # After adding class-specific transformation
        if self.verb_class == "1": # guna form
            return self.get_verb_grade(1).string
        elif self.verb_class == "4":
            returnString = DevanagariString(self.root) + DevanagariString("य्")
            return returnString.string
        elif self.verb_class == "6": # no modifications at all, otherwise similar to class 1
            return returnString.string
        elif self.verb_class == "10": # No reliable way to derive the base from the root
            returnString = self.get_verb_grade(self.verb_grade) + DevanagariString("अय्")
            return returnString.string

    def __init__(self, root, verb_class, verb_p_or_a, verb_base=None, verb_grade=None):
        """
        root: the root of the verb
        verb_class: the class of the verb (1, 4, 6, 10)
        verb_p_or_a: the voice of the verb (P, A)
        verb_base (optional): the base of the verb (especially if cannot be derived from the root). Eg: गच्छ् for गम्
        """
        super().__init__(root, verb_class, verb_p_or_a)

        self.verb_grade = verb_grade
        assert self.verb_class in ["1", "4", "6", "10"]
        
        if self.verb_class == "10":
            assert self.verb_grade in {0, 1, 2}, "Verb grade in {0,1,2} must be provided for class 10 verbs."

        if verb_base:
            self.verb_base = verb_base
        else:
            self.verb_base = self.get_verb_base()

    def get_verb_form(self, tense, number, person, passive=False):

        assert tense in self.tenses
        assert number in self.numbers
        assert person in self.persons

        if passive: # Passive will fail for ै roots
            return DevanagariString(self.prefixes[tense]) + DevanagariString(self.root) + "य्" + DevanagariString(self.suffixes["A"][tense][person+number])
        else:
            return DevanagariString(self.prefixes[tense]) + DevanagariString(self.verb_base) + DevanagariString(self.suffixes[self.verb_p_or_a][tense][person+number])
    
    def print_conjugation_tables(self, passive=False):
        print ("Verb class", self.verb_class, self.verb_p_or_a, end=" ")
        if passive:
            print ("(Passive)")
        else:
            print ()
        print ("Root:", self.root, "Base:", self.verb_base)
        print ()
        for tense in ["Present", "Imperfect", "Imperative", "Potential"]:
            print(tense)
            for person in ["1", "2", "3"]:
                for number in ["s", "d", "p"]:
                    print(self.get_verb_form(tense, number, person, passive), end="\t")
                print()
            print ()

class AthematicVerb(Verb):

    def get_verb_base(self, weak=False): # After adding class-specific transformation
        if self.verb_class == "2":
            if weak:
                return self.get_verb_grade(self.verb_grade_weak).string
            else:
                return self.get_verb_grade(self.verb_grade_strong).string
             
    
    def __init__(self, root, verb_class, verb_p_or_a, 
                 verb_base_weak=None,
                 verb_base_strong=None,
                 verb_grade_weak=None, 
                 verb_grade_strong=None):
        """
        root: the root of the verb
        verb_class: the class of the verb (1, 4, 6, 10)
        verb_p_or_a: the voice of the verb (P, A)
        verb_base (optional): the base of the verb (especially if cannot be derived from the root). Eg: गच्छ् for गम्
        """
        super().__init__(root, verb_class, verb_p_or_a)

        assert self.verb_class in ["2"]# , "3", "5", "7", "8", "9"]
        assert self.verb_grade_weak in {0, 1}
        assert self.verb_grade_strong in {1, 2}

        if verb_base_weak:
            self.verb_base_weak = verb_base_weak
        else:
            self.verb_base_weak = self.get_verb_base(weak=True)

        if verb_base_strong:
            self.verb_base_strong = verb_base_strong
        else:
            self.verb_base_strong = self.get_verb_base()










In [55]:
verb = ThematicVerb("विद्", "1", "P")
verb.print_conjugation_tables()

Verb class 1 P 
Root: विद् Base: वेद्

Present
वेदामि	वेदावः	वेदामः	
वेदसि	वेदथ	वेदथ	
वेदति	वेदतः	वेदन्ति	

Imperfect
अवेदम्	अवेदाव	अवेदाम	
अवेदः	अवेदतम्	अवेदत	
अवेदत्	अवेदताम्	अवेदन्	

Imperative
वेदानि	वेदाव	वेदाम	
वेद	वेदतम्	वेदत	
वेदतु	वेदताम्	वेदन्तु	

Potential
वेदेयम्	वेदेव	वेदेम	
वेदे:	वेदेतम्	वेदेत	
वेदेत्	वेदेताम्	वेदेयुः	



In [56]:
verb = ThematicVerb("विद्", "4", "A")
verb.print_conjugation_tables()

Verb class 4 A 
Root: विद् Base: विद्य्

Present
विद्ये	विद्यावहि	विद्यामहि	
विद्यसे	विद्येथे	विद्याध्वम्	
विद्यते	विद्येते	विद्यन्ते	

Imperfect
अविद्ये	अविद्यावहि	अविद्यामहि	
अविद्यथाः	अविद्येथाम्	अविद्यध्वम्	
अविद्यत	अविद्येताम्	अविद्यन्त	

Imperative
विद्यै	विद्यावहै	विद्यामहै	
विद्यस्व	विद्येथाम्	विद्यध्वम्	
विद्यताम्	विद्येताम्	विद्यन्ताम्	

Potential
विद्येय	विद्येवहि	विद्येमहि	
विद्येथाः	विद्येयाथाम्	विद्येध्वम्	
विद्येत	विद्येयताम्	विद्येरन्	



In [57]:
verb = ThematicVerb("भू", "1", "P")
verb.print_conjugation_tables()

Verb class 1 P 
Root: भू Base: भव्

Present
भवामि	भवावः	भवामः	
भवसि	भवथ	भवथ	
भवति	भवतः	भवन्ति	

Imperfect
अभवम्	अभवाव	अभवाम	
अभवः	अभवतम्	अभवत	
अभवत्	अभवताम्	अभवन्	

Imperative
भवानि	भवाव	भवाम	
भव	भवतम्	भवत	
भवतु	भवताम्	भवन्तु	

Potential
भवेयम्	भवेव	भवेम	
भवे:	भवेतम्	भवेत	
भवेत्	भवेताम्	भवेयुः	



In [58]:
verb = ThematicVerb("क्षम्", "4", "A")
verb.print_conjugation_tables()

Verb class 4 A 
Root: क्षम् Base: क्षम्य्

Present
क्षम्ये	क्षम्यावहि	क्षम्यामहि	
क्षम्यसे	क्षम्येथे	क्षम्याध्वम्	
क्षम्यते	क्षम्येते	क्षम्यन्ते	

Imperfect
अक्षम्ये	अक्षम्यावहि	अक्षम्यामहि	
अक्षम्यथाः	अक्षम्येथाम्	अक्षम्यध्वम्	
अक्षम्यत	अक्षम्येताम्	अक्षम्यन्त	

Imperative
क्षम्यै	क्षम्यावहै	क्षम्यामहै	
क्षम्यस्व	क्षम्येथाम्	क्षम्यध्वम्	
क्षम्यताम्	क्षम्येताम्	क्षम्यन्ताम्	

Potential
क्षम्येय	क्षम्येवहि	क्षम्येमहि	
क्षम्येथाः	क्षम्येयाथाम्	क्षम्येध्वम्	
क्षम्येत	क्षम्येयताम्	क्षम्येरन्	



In [59]:
verb2 = ThematicVerb("वन्द्", "1", "A")
verb2.print_conjugation_tables()

Verb class 1 A 
Root: वन्द् Base: वन्द्

Present
वन्दे	वन्दावहि	वन्दामहि	
वन्दसे	वन्देथे	वन्दाध्वम्	
वन्दते	वन्देते	वन्दन्ते	

Imperfect
अवन्दे	अवन्दावहि	अवन्दामहि	
अवन्दथाः	अवन्देथाम्	अवन्दध्वम्	
अवन्दत	अवन्देताम्	अवन्दन्त	

Imperative
वन्दै	वन्दावहै	वन्दामहै	
वन्दस्व	वन्देथाम्	वन्दध्वम्	
वन्दताम्	वन्देताम्	वन्दन्ताम्	

Potential
वन्देय	वन्देवहि	वन्देमहि	
वन्देथाः	वन्देयाथाम्	वन्देध्वम्	
वन्देत	वन्देयताम्	वन्देरन्	



In [60]:
verb = ThematicVerb("गै", "1", "P")
verb.print_conjugation_tables()

Verb class 1 P 
Root: गै Base: गाय्

Present
गायामि	गायावः	गायामः	
गायसि	गायथ	गायथ	
गायति	गायतः	गायन्ति	

Imperfect
अगायम्	अगायाव	अगायाम	
अगायः	अगायतम्	अगायत	
अगायत्	अगायताम्	अगायन्	

Imperative
गायानि	गायाव	गायाम	
गाय	गायतम्	गायत	
गायतु	गायताम्	गायन्तु	

Potential
गायेयम्	गायेव	गायेम	
गाये:	गायेतम्	गायेत	
गायेत्	गायेताम्	गायेयुः	



In [61]:
verb = ThematicVerb("गम्", "1", "P", verb_base='गच्छ्')
verb.print_conjugation_tables(passive=True)

Verb class 1 P (Passive)
Root: गम् Base: गच्छ्

Present
गम्ये	गम्यावहि	गम्यामहि	
गम्यसे	गम्येथे	गम्याध्वम्	
गम्यते	गम्येते	गम्यन्ते	

Imperfect
अगम्ये	अगम्यावहि	अगम्यामहि	
अगम्यथाः	अगम्येथाम्	अगम्यध्वम्	
अगम्यत	अगम्येताम्	अगम्यन्त	

Imperative
गम्यै	गम्यावहै	गम्यामहै	
गम्यस्व	गम्येथाम्	गम्यध्वम्	
गम्यताम्	गम्येताम्	गम्यन्ताम्	

Potential
गम्येय	गम्येवहि	गम्येमहि	
गम्येथाः	गम्येयाथाम्	गम्येध्वम्	
गम्येत	गम्येयताम्	गम्येरन्	



In [64]:
verb = ThematicVerb("चुर्", "10", "P", verb_grade=1)
verb.print_conjugation_tables()

Verb class 10 P 
Root: चुर् Base: चोरय्

Present
चोरयामि	चोरयावः	चोरयामः	
चोरयसि	चोरयथ	चोरयथ	
चोरयति	चोरयतः	चोरयन्ति	

Imperfect
अचोरयम्	अचोरयाव	अचोरयाम	
अचोरयः	अचोरयतम्	अचोरयत	
अचोरयत्	अचोरयताम्	अचोरयन्	

Imperative
चोरयानि	चोरयाव	चोरयाम	
चोरय	चोरयतम्	चोरयत	
चोरयतु	चोरयताम्	चोरयन्तु	

Potential
चोरयेयम्	चोरयेव	चोरयेम	
चोरये:	चोरयेतम्	चोरयेत	
चोरयेत्	चोरयेताम्	चोरयेयुः	



In [65]:
verb = ThematicVerb("मन्त्र्", "10", "A", verb_grade=1)
verb.print_conjugation_tables()

Verb class 10 A 
Root: मन्त्र् Base: मन्त्रय्

Present
मन्त्रये	मन्त्रयावहि	मन्त्रयामहि	
मन्त्रयसे	मन्त्रयेथे	मन्त्रयाध्वम्	
मन्त्रयते	मन्त्रयेते	मन्त्रयन्ते	

Imperfect
अमन्त्रये	अमन्त्रयावहि	अमन्त्रयामहि	
अमन्त्रयथाः	अमन्त्रयेथाम्	अमन्त्रयध्वम्	
अमन्त्रयत	अमन्त्रयेताम्	अमन्त्रयन्त	

Imperative
मन्त्रयै	मन्त्रयावहै	मन्त्रयामहै	
मन्त्रयस्व	मन्त्रयेथाम्	मन्त्रयध्वम्	
मन्त्रयताम्	मन्त्रयेताम्	मन्त्रयन्ताम्	

Potential
मन्त्रयेय	मन्त्रयेवहि	मन्त्रयेमहि	
मन्त्रयेथाः	मन्त्रयेयाथाम्	मन्त्रयेध्वम्	
मन्त्रयेत	मन्त्रयेयताम्	मन्त्रयेरन्	



In [66]:
verb = ThematicVerb("विद्", "1", "P")
verb.print_conjugation_tables(passive=True)

Verb class 1 P (Passive)
Root: विद् Base: वेद्

Present
विद्ये	विद्यावहि	विद्यामहि	
विद्यसे	विद्येथे	विद्याध्वम्	
विद्यते	विद्येते	विद्यन्ते	

Imperfect
अविद्ये	अविद्यावहि	अविद्यामहि	
अविद्यथाः	अविद्येथाम्	अविद्यध्वम्	
अविद्यत	अविद्येताम्	अविद्यन्त	

Imperative
विद्यै	विद्यावहै	विद्यामहै	
विद्यस्व	विद्येथाम्	विद्यध्वम्	
विद्यताम्	विद्येताम्	विद्यन्ताम्	

Potential
विद्येय	विद्येवहि	विद्येमहि	
विद्येथाः	विद्येयाथाम्	विद्येध्वम्	
विद्येत	विद्येयताम्	विद्येरन्	



In [67]:
verb = ThematicVerb("भू", "1", "P")
verb.print_conjugation_tables(passive=True)

Verb class 1 P (Passive)
Root: भू Base: भव्

Present
भूये	भूयावहि	भूयामहि	
भूयसे	भूयेथे	भूयाध्वम्	
भूयते	भूयेते	भूयन्ते	

Imperfect
अभूये	अभूयावहि	अभूयामहि	
अभूयथाः	अभूयेथाम्	अभूयध्वम्	
अभूयत	अभूयेताम्	अभूयन्त	

Imperative
भूयै	भूयावहै	भूयामहै	
भूयस्व	भूयेथाम्	भूयध्वम्	
भूयताम्	भूयेताम्	भूयन्ताम्	

Potential
भूयेय	भूयेवहि	भूयेमहि	
भूयेथाः	भूयेयाथाम्	भूयेध्वम्	
भूयेत	भूयेयताम्	भूयेरन्	

